***Reading file***

In [8]:
import numpy as np
import pandas as pd
import json
from transformers import pipeline
import numpy as np
import torch
from transformers import BertModel, BertTokenizer
from torch.nn.functional import normalize

In [9]:
json_file = "merged_data_10000_ours.json"

data = []
with open(json_file, 'r') as file:
    for line in file:
        data.append(json.loads(line))

train_df = pd.DataFrame(data)

In [10]:
train_df

,video_id,question_id,type,question_content,templ_values,question_deleted,anser,question,Quest_hindi,ans_hindi
0,00000136,17,"[""Audio-Visual"", ""Comparative""]",Is the instrument on the <LR> louder than the ...,"[""left"", ""right""]",0,no,Is the instrument on the left louder than the ...,क्या बायीं ओर का वाद्ययंत्र दाहिनी ओर के वाद्य...,नहीं
1,00000250,23,"[""Audio-Visual"", ""Existential""]",Is the <Object> in the video always playing?,"[""accordion""]",0,yes,Is the accordion in the video always playing?,क्या वीडियो में अकॉर्डियन हमेशा बजता रहता है?,हाँ
2,00000225,27,"[""Audio-Visual"", ""Existential""]",Is there a voiceover?,[],0,yes,Is there a voiceover?,क्या कोई वॉयसओवर है?,हाँ
3,00000152,30,"[""Audio-Visual"", ""Existential""]",Is this sound from the instrument in the video?,[],0,yes,Is this sound from the instrument in the video?,क्या यह ध्वनि वीडियो में मौजूद उपकरण की है?,हाँ
4,00000085,32,"[""Audio-Visual"", ""Existential""]",Is there a voiceover?,[],0,no,Is there a voiceover?,क्या कोई वॉयसओवर है?,नहीं
...,...,...,...,...,...,...,...,...,...,...
9995,00003985,28445,"[""Visual"", ""Counting""]",Are there <Object> and <Object> instruments in...,"[""banjo"", ""violin""]",0,yes,Are there banjo and violin instruments in the ...,क्या वीडियो में बैंजो और वायलिन वाद्ययंत्र हैं?,हाँ
9996,00000326,28446,"[""Audio"", ""Comparative""]",Is the <Object> playing longer than the <Object>?,"[""piano"", ""violin""]",0,no,Is the piano playing longer than the violin?,क्या पियानो वायलिन से अधिक देर तक बजता है?,नहीं
9997,00003730,28458,"[""Audio-Visual"", ""Comparative""]",Is the <Object> on the <LR> more rhythmic than...,"[""cello"", ""left"", ""ukulele"", ""right""]",0,no,Is the cello on the left more rhythmic than th...,क्या बायीं ओर का सेलो दायीं ओर के युकुलेले की ...,नहीं
9998,00008427,28462,"[""Visual"", ""Counting""]",Are there <Object> and <Object> instruments in...,"[""violin"", ""cello""]",0,yes,Are there violin and cello instruments in the ...,क्या वीडियो में वायलिन और सेलो वाद्ययंत्र हैं?,हाँ


***English2Hingish***

In [ ]:
# Initialize the Hinglish translation pipeline
model_name = "SkAndMl/english-to-hinglish"
pipe = pipeline("text2text-generation", model=model_name)

# Function to translate English to Hinglish
def translate_to_hinglish(text):
    return pipe(text)[0]['generated_text']

train_df['hinglish_question'] = train_df['question'].apply(translate_to_hinglish)

print(train_df[['question', 'hinglish_question']])

In [12]:
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_df

***Hinglish embeddings***

In [16]:
def get_average_pooling_embedding(text):
    # Tokenize input text
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    # Get model output
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Extract embeddings
    embeddings = model_output.last_hidden_state
    # Perform average pooling across the sequence dimension
    input_mask_expanded = encoded_input['attention_mask'].unsqueeze(-1).expand(embeddings.size()).float()
    sum_embeddings = torch.sum(embeddings * input_mask_expanded, 1)
    sum_mask = input_mask_expanded.sum(1)
    sum_mask = torch.clamp(sum_mask, min=1e-9)
    mean_embeddings = sum_embeddings / sum_mask
    return mean_embeddings

# Assuming train_df['hinglish_question'] exists and contains the text entries
embeddings_list = []

for text in train_df['hinglish_question']:
    embedding = get_average_pooling_embedding(text).cpu().numpy()  # Convert to NumPy array and ensure moving to CPU if necessary
    embeddings_list.append(embedding.squeeze())  # Remove the singleton dimension

# Convert the list of embeddings to a numpy array
embeddings_array = np.vstack(embeddings_list)  # This will create a 2D array

# Save the numpy array to a .npy file
np.save('hingligh_question_embeddings.npy', embeddings_array)


In [17]:
train_df.to_json('merged_data_with_hinglish.json', orient='records', lines=True)


***English Embeddings***

In [9]:
def get_average_pooling_embedding(text):
    # Tokenize input text
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    # Get model output
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Extract embeddings
    embeddings = model_output.last_hidden_state
    # Perform average pooling across the sequence dimension
    input_mask_expanded = encoded_input['attention_mask'].unsqueeze(-1).expand(embeddings.size()).float()
    sum_embeddings = torch.sum(embeddings * input_mask_expanded, 1)
    sum_mask = input_mask_expanded.sum(1)
    sum_mask = torch.clamp(sum_mask, min=1e-9)
    mean_embeddings = sum_embeddings / sum_mask
    return mean_embeddings

# Assuming train_df['hinglish_question'] exists and contains the text entries
embeddings_list = []

for text in train_df['question']:
    embedding = get_average_pooling_embedding(text).cpu().numpy()  # Convert to NumPy array and ensure moving to CPU if necessary
    embeddings_list.append(embedding.squeeze())  # Remove the singleton dimension

# Convert the list of embeddings to a numpy array
embeddings_array = np.vstack(embeddings_list)  # This will create a 2D array

# Save the numpy array to a .npy file
np.save('english_question_embeddings.npy', embeddings_array)


***Hindi Embeddigs***

In [13]:
def get_average_pooling_embedding(text):
    # Tokenize input text
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    # Get model output
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Extract embeddings
    embeddings = model_output.last_hidden_state
    # Perform average pooling across the sequence dimension
    input_mask_expanded = encoded_input['attention_mask'].unsqueeze(-1).expand(embeddings.size()).float()
    sum_embeddings = torch.sum(embeddings * input_mask_expanded, 1)
    sum_mask = input_mask_expanded.sum(1)
    sum_mask = torch.clamp(sum_mask, min=1e-9)
    mean_embeddings = sum_embeddings / sum_mask
    return mean_embeddings

# Assuming train_df['hinglish_question'] exists and contains the text entries
embeddings_list = []

for text in train_df['Quest_hindi']:
    embedding = get_average_pooling_embedding(text).cpu().numpy()  # Convert to NumPy array and ensure moving to CPU if necessary
    embeddings_list.append(embedding.squeeze())  # Remove the singleton dimension

# Convert the list of embeddings to a numpy array
embeddings_array = np.vstack(embeddings_list)  # This will create a 2D array

# Save the numpy array to a .npy file
np.save('hindi_question_embeddings.npy', embeddings_array)
